In [2]:
import numpy as np
import pandas as pd
import re


In [3]:
class Layer:
    def __init__(self, number, size, inputs=1):
        self.number = number
        self.size = size
        self.data = np.zeros((inputs, size), dtype=np.float64)

class Weight:
    def __init__(self, prev, next):
        self.value = (np.random.randn(prev.size, next.size).astype(np.float64))

class Bias:
    def __init__(self, next):
        self.value = np.random.randn(1, next.size).astype(np.float64)

class Neural:
    def __init__(self, activation):
        self.activation = activation
        self.layers = []
        self.weights = []
        self.biases = []

    def add_layer(self, layer):
        self.layers.append(layer)
        if len(self.layers) > 1:
            self.weights.append(Weight(self.layers[-2], self.layers[-1]))
            self.biases.append(Bias(self.layers[-1]))

    def forward(self):
        self.z_values = []
        self.a_values = []
        for i in range(1, len(self.layers)):
            prev_data = self.layers[i - 1].data
            w = self.weights[i - 1].value
            b = self.biases[i - 1].value
            z = np.dot(prev_data, w) + b
            self.z_values.append(z)
            a = self.activation(z)
            self.a_values.append(a)
            self.layers[i].data = a

    @property
    def output(self):
        return self.layers[-1].data

def sigmoid(x):
    x = np.clip(x, -500, 500)
    return 1 / (1 + np.exp(-x))

def costFunction(predicted, real):
    return np.sum((predicted - real) ** 2)



In [ ]:
# ---------------------- Text Processor ---------------------- #
class TextProcessor:
    def __init__(self):
        self.vocab = {}

    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r"<.*?>", "", text)
        text = re.sub(r"[^a-zA-Z']", " ", text)
        return text

    def build_vocab(self, texts):
        words = set()
        for sentence in texts:
            words.update(sentence.split())
        self.vocab = {word: i for i, word in enumerate(words)}

    def vectorize(self, sentence):
        vec = np.zeros(len(self.vocab))
        for word in sentence.split():
            if word in self.vocab:
                vec[self.vocab[word]] += 1
        return vec

    def process(self, df, text_column):
        df[text_column] = df[text_column].apply(self.clean_text)
        self.build_vocab(df[text_column])
        vectors = np.array([self.vectorize(text) for text in df[text_column]])
        return vectors / np.max(vectors)
        


In [5]:
df = pd.read_csv("IMDB Dataset.csv")
df = df[:2000]
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

processor = TextProcessor()
X = processor.process(df, 'review')
y = df['sentiment'].values.reshape(-1, 1)

model = Neural(sigmoid)
model.add_layer(Layer(0, X.shape[1], inputs=X.shape[0]))
model.layers[0].data = X
model.add_layer(Layer(1, 32, inputs=X.shape[0]))
model.add_layer(Layer(2, 16, inputs=X.shape[0]))
model.add_layer(Layer(3, 1, inputs=X.shape[0]))


In [6]:
# Training 
epochs = 100
lr = 0.1

for epoch in range(epochs):
    model.forward()
    predicted = model.output
    loss = costFunction(predicted, y)
    dL = 2 * (predicted - y) / y.shape[0]

    for i in reversed(range(len(model.weights))):
        z = model.z_values[i]
        a_prev = model.a_values[i - 1] if i > 0 else model.layers[0].data
        dz = dL * model.a_values[i] * (1 - model.a_values[i])

        dw = np.dot(a_prev.T, dz)
        db = np.sum(dz, axis=0, keepdims=True)

        model.weights[i].value -= lr * dw
        model.biases[i].value -= lr * db

        dL = np.dot(dz, model.weights[i].value.T)

    print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")


Epoch 1, Loss: 924.0395
Epoch 2, Loss: 922.0602
Epoch 3, Loss: 919.9797
Epoch 4, Loss: 917.7907
Epoch 5, Loss: 915.4852
Epoch 6, Loss: 913.0545
Epoch 7, Loss: 910.4893
Epoch 8, Loss: 907.7792
Epoch 9, Loss: 904.9130
Epoch 10, Loss: 901.8782
Epoch 11, Loss: 898.6613
Epoch 12, Loss: 895.2477
Epoch 13, Loss: 891.6210
Epoch 14, Loss: 887.7635
Epoch 15, Loss: 883.6559
Epoch 16, Loss: 879.2768
Epoch 17, Loss: 874.6032
Epoch 18, Loss: 869.6097
Epoch 19, Loss: 864.2688
Epoch 20, Loss: 858.5509
Epoch 21, Loss: 852.4238
Epoch 22, Loss: 845.8532
Epoch 23, Loss: 838.8025
Epoch 24, Loss: 831.2336
Epoch 25, Loss: 823.1068
Epoch 26, Loss: 814.3819
Epoch 27, Loss: 805.0197
Epoch 28, Loss: 794.9829
Epoch 29, Loss: 784.2391
Epoch 30, Loss: 772.7632
Epoch 31, Loss: 760.5412
Epoch 32, Loss: 747.5748
Epoch 33, Loss: 733.8860
Epoch 34, Loss: 719.5226
Epoch 35, Loss: 704.5636
Epoch 36, Loss: 689.1227
Epoch 37, Loss: 673.3506
Epoch 38, Loss: 657.4332
Epoch 39, Loss: 641.5863
Epoch 40, Loss: 626.0449
Epoch 41,

In [8]:

text = "It was great product."
cleaned = processor.clean_text(text)
vec = processor.vectorize(cleaned).reshape(1, -1)
vec = vec / np.max(vec)
model.layers[0].data = vec
model.forward()
output = model.output[0][0]
print("Sentiment:", "Positive" if output > 0.5 else "Negative")


Sentiment: Negative
